In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "MH"
BARK_VAR = "WC"
FIA_SPCD = 264

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(0.0, 0.94967, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,4123.000000,4.123000e+03,4123.0,4123.000000,4123.000000,4123.000000,4123.000000,4123.000000,4123.000000,4123.000000,...,141.000000,141.000000,4123.000000,4123.000000,4123.000000,4123.000000,4123.0,4123.000000,4123.000000,4123.000000
mean,596.586951,4.052218e+11,264.0,55.014876,0.292508,3.109531,57.623333,15.111521,0.576539,0.562006,...,0.165248,0.333333,2005.366723,8.167839,2015.366723,7.991754,10.0,2.780015,10.749212,273.875819
std,43.382655,1.381559e+11,0.0,36.094264,0.224943,2.059593,13.894540,9.806877,0.189249,0.187621,...,0.113852,0.235104,2.438049,1.114922,2.438049,1.115995,0.0,2.033291,5.831673,139.185844
min,113.000000,6.020896e+10,264.0,2.800000,0.000000,0.000000,19.000000,5.000000,0.020000,0.020000,...,0.050000,0.050000,2001.000000,5.000000,2011.000000,4.000000,10.0,0.000000,0.000000,0.000000
25%,601.000000,4.101040e+11,264.0,29.244000,0.120000,1.221730,49.000000,7.200000,0.450000,0.400000,...,0.100000,0.150000,2003.000000,7.000000,2013.000000,7.000000,10.0,1.000000,6.000000,163.000000
50%,606.000000,4.103018e+11,264.0,42.800000,0.200000,3.176499,57.000000,11.500000,0.600000,0.560000,...,0.150000,0.300000,2005.000000,8.000000,2015.000000,8.000000,10.0,2.000000,12.000000,279.000000
75%,617.000000,5.305058e+11,264.0,74.762000,0.420000,5.096361,63.000000,21.000000,0.700000,0.700000,...,0.200000,0.400000,2008.000000,9.000000,2018.000000,9.000000,10.0,5.000000,15.000000,390.000000
max,710.000000,5.309064e+11,264.0,195.000000,1.550000,6.283185,99.000000,61.600000,0.990000,0.990000,...,0.600000,1.250000,2009.000000,11.000000,2019.000000,11.000000,10.0,8.000000,27.000000,510.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [13:53<00:00,  2.40it/s, 255 steps of size 2.56e-02. a


Done.
Samples saved at ../../models/maicf/MH//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/MH//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [25:42<00:00,  1.30it/s, 255 steps of size 2.04e-02. a


Done.
Samples saved at ../../models/maicf/MH//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/MH//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/MH//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/MH//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [12:54<00:00,  2.58it/s, 255 steps of size 2.63e-02. a


Done.
Samples saved at ../../models/maicf/MH//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/MH//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [25:21<00:00,  1.31it/s, 255 steps of size 1.82e-02. a


Done.
Samples saved at ../../models/maicf/MH//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/MH//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [10:25<00:00,  3.20it/s, 255 steps of size 2.47e-02. a


Done.
Samples saved at ../../models/maicf/MH//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/MH//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [10:12<00:00,  3.27it/s, 255 steps of size 2.62e-02. a


Done.
Samples saved at ../../models/maicf/MH//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/MH//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/MH//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/MH//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [18:21<00:00,  1.82it/s, 255 steps of size 2.07e-02. a


Done.
Samples saved at ../../models/maicf/MH//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/MH//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [18:47<00:00,  1.77it/s, 255 steps of size 1.98e-02. a


Done.
Samples saved at ../../models/maicf/MH//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/MH//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
